# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [1]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus
!pip install spacy dateparser
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 43.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

KeyboardInterrupt: Interrupted by user

## 📦 Setup and Configuration
Import required packages and setup environment.

In [5]:
import os
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pyinputplus as pyip
from datetime import datetime, timedelta
import json
import re

# Optional: if your environment provides AI or external packages
try:
    from fetch_my_weather import get_weather
    WEATHER_AVAILABLE = True
except ImportError:
    WEATHER_AVAILABLE = False
    print("fetch-my-weather not available, will use wttr.in API")

# Matplotlib styling (works with or without seaborn installed)
if 'seaborn-v0_8' in plt.style.available:
    plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)


## 🌤️ Weather Data Functions

In [4]:
import json
import requests
from urllib.parse import quote

def _safe(d, *keys, default=None):
    """Safe nested get for dict/list (avoids KeyError/IndexError)."""
    cur = d
    for k in keys:
        if isinstance(cur, dict) and k in cur:
            cur = cur[k]
        elif isinstance(cur, list) and isinstance(k, int) and 0 <= k < len(cur):
            cur = cur[k]
        else:
            return default
    return cur

def get_weather_data(location, forecast_days=3):
    """
    Retrieve weather data using wttr.in JSON (3-day forecast).
    Relaxed validation: show data for any resolvable input.
    Only errors: network/HTTP/invalid JSON or missing forecast.
    """
    if not isinstance(location, str) or not location.strip():
        return {"error": "Location must be a non-empty string.", "location": location}

    user_input = location.strip()
    try:
        days = max(1, min(int(forecast_days), 3))  # wttr.in offers up to 3 days
    except Exception:
        days = 3

    url = f"https://wttr.in/{quote(user_input)}?format=j1"
    try:
        resp = requests.get(url, timeout=10, headers={"User-Agent": "WeatherWise/1.0"})
        # If wttr.in truly can’t resolve, it may return 404.
        if resp.status_code == 404:
            return {"error": f"No weather found for '{user_input}'.", "location": user_input}
        resp.raise_for_status()
        data = resp.json()  # may raise JSONDecodeError
    except requests.exceptions.Timeout:
        return {"error": "Timeout contacting weather service.", "location": user_input}
    except requests.exceptions.HTTPError as e:
        return {"error": f"Weather service error: {e.response.status_code}", "location": user_input}
    except json.JSONDecodeError:
        return {"error": "Weather service returned invalid JSON.", "location": user_input}
    except Exception as e:
        return {"error": f"Could not retrieve weather data: {e}", "location": user_input}

    # Normalize fields (accept suburb/city/region that wttr.in returns)
    nearest  = _safe(data, "nearest_area", 0, default={}) or {}
    current  = _safe(data, "current_condition", 0, default={}) or {}
    forecast = _safe(data, "weather", default=[]) or []

    if not isinstance(forecast, list) or len(forecast) == 0:
        return {"error": f"No forecast data available for '{user_input}'.", "location": user_input}

    resolved_location = (
        _safe(nearest, "areaName", 0, "value", default=user_input)
        or user_input
    )

    # Return normalized structure your charts/UI expect
    return {
        "source": "wttr.in",
        "requested_location": user_input,     # what the user typed
        "location": resolved_location,        # wttr.in’s best match (may be a suburb)
        "display_location": user_input,       # show the user’s input in titles/UI
        "current": current,
        "forecast": forecast[:days],
        "raw": data,
    }


## 📊 Visualisation Functions

In [3]:
import math
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# ===== Helper functions =====
def _daily_series_from_wttrin(weather_data):
    """Extract daily values: dates, hiC, loC, avg_precip, avg_pop."""
    days = weather_data.get("forecast", [])
    dates, hiC, loC, avg_precip, avg_pop = [], [], [], [], []
    for d in days:
        dt = datetime.strptime(d["date"], "%Y-%m-%d")
        dates.append(dt)
        hi = d.get("maxtempC"); lo = d.get("mintempC")
        hiC.append(float(hi) if hi is not None else math.nan)
        loC.append(float(lo) if lo is not None else math.nan)

        hrs = d.get("hourly", []) or []
        if hrs:
            mm  = [float(h.get("precipMM", 0)) for h in hrs]
            pop = [float(h.get("chanceofrain", 0)) for h in hrs if h.get("chanceofrain") is not None]
            avg_precip.append(sum(mm)/len(mm) if mm else 0.0)
            avg_pop.append(sum(pop)/len(pop) if pop else math.nan)
        else:
            avg_precip.append(math.nan); avg_pop.append(math.nan)
    return dates, hiC, loC, avg_precip, avg_pop


def _title_loc_and_range(weather_data, dates):
    loc = weather_data.get("display_location", weather_data.get("requested_location", "Your city"))
    if dates:
        drange = f"{dates[0].strftime('%d %b')} – {dates[-1].strftime('%d %b')}"
        return f"{loc} · {drange}"
    return loc


# ===== Temperature =====
def create_temperature_visualisation(weather_data, output_type='display'):
    # Basic structure checks
    if not isinstance(weather_data, dict) or "forecast" not in weather_data:
        print("No weather data to visualize."); return
    dates, hiC, loC, _, _ = _daily_series_from_wttrin(weather_data)
    if not dates:
        print("No forecast data to visualize."); return

    # Do we have any finite temperature values?
    has_hi = any(isinstance(v, (int,float)) and math.isfinite(v) for v in hiC)
    has_lo = any(isinstance(v, (int,float)) and math.isfinite(v) for v in loC)
    if not (has_hi or has_lo):
        # Graceful, visible fallback so you know why nothing showed
        fig, ax = plt.subplots()
        ax.set_title(f"Temperatures — {_title_loc_and_range(weather_data, dates)}")
        ax.text(0.5, 0.5, "No temperature data from provider.", ha="center", va="center",
                transform=ax.transAxes)
        ax.axis("off")
        if output_type == 'figure': return fig
        plt.show()
        return

    # Rolling mean of mid temps (3-pt window), robust to NaNs
    mids = []
    for h, l in zip(hiC, loC):
        m = (h + l) / 2 if (isinstance(h,(int,float)) and isinstance(l,(int,float))
                            and math.isfinite(h) and math.isfinite(l)) else math.nan
        mids.append(m)
    roll = []
    for i in range(len(mids)):
        win = mids[max(0, i-1):min(len(mids), i+2)]
        vals = [v for v in win if isinstance(v,(int,float)) and math.isfinite(v)]
        roll.append(sum(vals)/len(vals) if vals else math.nan)

    fig, ax = plt.subplots()

    # Comfort band + weekend shading
    ax.axhspan(18, 26, alpha=0.12)
    for dt in dates:
        if dt.weekday() in (5, 6):
            ax.axvspan(dt, dt, alpha=0.08)

    # High/low + band + rolling mean (Matplotlib will skip NaNs)
    if has_hi:
        ax.plot(dates, hiC, marker="o", markersize=7, linewidth=2, label="High")
    if has_lo:
        ax.plot(dates, loC, marker="o", markersize=7, linewidth=2, label="Low")
    if has_hi and has_lo:
        ax.fill_between(dates, loC, hiC, alpha=0.25, linewidth=0)
    # Only plot rolling if at least one finite value exists
    if any(isinstance(v,(int,float)) and math.isfinite(v) for v in roll):
        ax.plot(dates, roll, linestyle="--", linewidth=2, label="Rolling mean")

    # Annotate extrema when available
    valid_hi_idx = [i for i,v in enumerate(hiC) if isinstance(v,(int,float)) and math.isfinite(v)]
    valid_lo_idx = [i for i,v in enumerate(loC) if isinstance(v,(int,float)) and math.isfinite(v)]
    if valid_hi_idx:
        i_max = max(valid_hi_idx, key=lambda i: hiC[i])
        ax.annotate(f"Max {hiC[i_max]:.0f}°C", (dates[i_max], hiC[i_max]),
                    xytext=(6, 10), textcoords="offset points")
    if valid_lo_idx:
        i_min = min(valid_lo_idx, key=lambda i: loC[i])
        ax.annotate(f"Min {loC[i_min]:.0f}°C", (dates[i_min], loC[i_min]),
                    xytext=(6, -16), textcoords="offset points")

    ax.set_title(f"Temperatures — {_title_loc_and_range(weather_data, dates)}")
    ax.set_xlabel("Date"); ax.set_ylabel("°C")
    if has_hi or has_lo:
        ax.legend()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d %b'))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.grid(True, which="major", axis="y", alpha=0.3)
    plt.xticks(rotation=0)
    plt.tight_layout()
    if output_type == 'figure': return fig
    plt.show()



# ===== Precipitation =====
def create_precipitation_visualisation(weather_data, output_type='display'):

    if not isinstance(weather_data, dict) or "forecast" not in weather_data:
        print("No weather data to visualize."); return
    days = weather_data.get("forecast", [])
    if not days:
        print("No forecast data to visualize."); return


    days = weather_data.get("forecast", [])
    if not days:
        print("No forecast data to visualize.")
        return

    dates, avg_precip, avg_pop = [], [], []
    for d in days:
        dt = datetime.strptime(d["date"], "%Y-%m-%d")
        hrs = d.get("hourly", []) or []
        mm  = [float(h.get("precipMM", 0)) for h in hrs]
        pop = [float(h.get("chanceofrain", 0)) for h in hrs if h.get("chanceofrain") is not None]
        dates.append(dt)
        avg_precip.append(sum(mm)/len(mm) if mm else 0.0)
        avg_pop.append(sum(pop)/len(pop) if pop else float("nan"))

    fig, ax = plt.subplots()
    ax.plot(dates, avg_precip, marker="o", markersize=6, linewidth=2, label="Avg precip (mm)")

    finite_pops = [p for p in avg_pop if isinstance(p, (int, float)) and math.isfinite(p)]
    has_pop = len(finite_pops) > 0

    if has_pop:
        max_precip = max(avg_precip) if avg_precip else 1.0
        max_pop = max(finite_pops) if finite_pops else 100.0
        target_peak_mm = max(max_precip, 1.0)
        scale = (target_peak_mm / max_pop) if max_pop > 0 else 0.01
        if not math.isfinite(scale) or scale <= 1e-9: scale = 0.01

        scaled_pop = [(p * scale) if math.isfinite(p) else float("nan") for p in avg_pop]
        ax.plot(dates, scaled_pop, marker="o", markersize=6, linewidth=2, label="Chance of rain (scaled)")

        # 50% line only (no emoji markers)
        thresh_y = 50.0 * scale
        ax.axhline(thresh_y, linestyle=":", linewidth=1)

        # Right axis for % values
        ax2 = ax.twinx()
        y0, y1 = ax.get_ylim()
        ax2.set_ylim(y0/scale, y1/scale)
        ax2.set_ylabel("% (right)")

    # Wettest-day tag
    if avg_precip:
        i_wet = max(range(len(avg_precip)), key=lambda i: avg_precip[i])
        ax.annotate(f"Wettest ~{avg_precip[i_wet]:.1f} mm", (dates[i_wet], avg_precip[i_wet]),
                    xytext=(6, 10), textcoords="offset points")

    ax.set_title(f"Precipitation & Rain Chance — {_title_loc_and_range(weather_data, dates)}")
    ax.set_xlabel("Date"); ax.set_ylabel("mm (left)")
    ax.grid(True, which="major", axis="y", alpha=0.3)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d %b'))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.legend(loc="upper right")
    plt.xticks(rotation=0)
    plt.tight_layout()
    if output_type == 'figure': return fig
    plt.show()


# ===== Humidity =====
def create_humidity_visualisation(weather_data, output_type='display'):

    if not isinstance(weather_data, dict) or "forecast" not in weather_data:
        print("No weather data to visualize."); return
    days = weather_data.get("forecast", [])
    if not days:
        print("No forecast data to visualize."); return

    days = weather_data.get("forecast", [])
    if not days:
        print("No forecast data to visualize.")
        return

    dates, avg_hum = [], []
    for d in days:
        try:
            dt = datetime.strptime(d["date"], "%Y-%m-%d")
        except Exception:
            continue
        hrs = d.get("hourly", []) or []
        hums = [float(h.get("humidity", 0)) for h in hrs if h.get("humidity") is not None]
        dates.append(dt)
        avg_hum.append(sum(hums)/len(hums) if hums else float("nan"))

    if not dates:
        print("No data to visualize.")
        return

    fig, ax = plt.subplots()
    ax.axhspan(30, 60, alpha=0.10)  # comfy humidity band
    for dt in dates:
        if dt.weekday() in (5, 6):
            ax.axvspan(dt, dt, alpha=0.08)

    ax.plot(dates, avg_hum, marker="o", linewidth=2)
    valid_idx = [i for i, h in enumerate(avg_hum) if math.isfinite(h)]
    if valid_idx:
        i_mug = max(valid_idx, key=lambda i: avg_hum[i])
        ax.annotate(f"Muggiest {avg_hum[i_mug]:.0f}%", (dates[i_mug], avg_hum[i_mug]),
                    xytext=(6, 10), textcoords="offset points")

    ax.set_title(f"Average Humidity — {_title_loc_and_range(weather_data, dates)}")
    ax.set_xlabel("Date"); ax.set_ylabel("%")
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d %b'))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.grid(True, which="major", axis="y", alpha=0.3)
    plt.xticks(rotation=0)
    plt.tight_layout()
    if output_type == 'figure': return fig
    plt.show()


# ===== Wind =====
def create_wind_visualisation(weather_data, output_type='display'):

    if not isinstance(weather_data, dict) or "forecast" not in weather_data:
        print("No weather data to visualize."); return
    days = weather_data.get("forecast", [])
    if not days:
        print("No forecast data to visualize."); return

    days = weather_data.get("forecast", [])
    if not days:
        print("No forecast data to visualize.")
        return

    dates, max_wind = [], []
    for d in days:
        try:
            dt = datetime.strptime(d["date"], "%Y-%m-%d")
        except Exception:
            continue
        hrs = d.get("hourly", []) or []
        winds = [float(h.get("windspeedKmph", 0)) for h in hrs if h.get("windspeedKmph") is not None]
        dates.append(dt)
        max_wind.append(max(winds) if winds else float("nan"))

    if not dates:
        print("No data to visualize.")
        return

    fig, ax = plt.subplots()
    ax.axhspan(20, 30, alpha=0.08)
    ax.axhspan(30, 50, alpha=0.10)
    ax.axhspan(50, 200, alpha=0.12)
    for dt in dates:
        if dt.weekday() in (5, 6):
            ax.axvspan(dt, dt, alpha=0.08)

    ax.plot(dates, max_wind, marker="o", linewidth=2)
    valid_idx = [i for i, v in enumerate(max_wind) if math.isfinite(v)]
    if valid_idx:
        i_peak = max(valid_idx, key=lambda i: max_wind[i])
        ax.annotate(f"Peak {max_wind[i_peak]:.0f} km/h", (dates[i_peak], max_wind[i_peak]),
                    xytext=(6, 10), textcoords="offset points")

    ax.set_title(f"Max Wind Speed — {_title_loc_and_range(weather_data, dates)}")
    ax.set_xlabel("Date"); ax.set_ylabel("km/h")
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d %b'))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.grid(True, which="major", axis="y", alpha=0.3)
    plt.xticks(rotation=0)
    plt.tight_layout()
    if output_type == 'figure': return fig
    plt.show()


## 🤖 Natural Language Processing

In [9]:
# ===================== SMART NLP PARSER (robust + fuzzy cities) =====================
import re, math, difflib
from datetime import datetime, timedelta

# Optional libs
try:
    import spacy
    from dateparser import parse as dateparse
    _NLP_OK = True
except Exception:
    spacy = None
    dateparse = None
    _NLP_OK = False

# Lazy spaCy loader
_NLP = None
def _get_nlp():
    global _NLP
    if not _NLP_OK:
        return None
    if _NLP is None:
        try:
            _NLP = spacy.load("en_core_web_sm")
        except Exception:
            try:
                from spacy.cli import download as spacy_download
                spacy_download("en_core_web_sm")
                _NLP = spacy.load("en_core_web_sm")
            except Exception:
                _NLP = None
    return _NLP

_WEEKDAYS = ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]
_TIME_WORDS = set(_WEEKDAYS + ["today","tomorrow","weekend","tonight","morning","afternoon","evening","next","this"])

# Small free city lexicon for fuzzy matching (edit freely to add more)
_CITY_LEXICON = {
    # AU
    "perth","sydney","melbourne","brisbane","adelaide","hobart","darwin","canberra","gold coast","newcastle",
    # NZ
    "auckland","wellington","christchurch","queenstown",
    # Asia / misc
    "kathmandu","singapore","tokyo","seoul","bangkok","jakarta",
    # EU / US (handful)
    "london","paris","berlin","madrid","rome","amsterdam","dublin",
    "new york","los angeles","san francisco","chicago","seattle","boston","houston",
}

def _clean_loc_piece(txt: str) -> str:
    """Trim trailing time words/punctuation from captured location."""
    if not txt: return txt
    # remove common trailing punctuation
    t = txt.strip(" ,.?;:!/-").lower()
    # drop trailing time tokens (e.g., 'sydey tomorrow' -> 'sydey')
    parts = [p for p in re.split(r"\s+", t) if p]
    while parts and parts[-1] in _TIME_WORDS:
        parts.pop()
    return " ".join(parts)

def _maybe_split_in_stuck(token: str):
    """
    Recover city from glued forms:
      - 'inkathmandu' -> ('in','kathmandu')
      - 'inperth'     -> ('in','perth')
    """
    m = re.match(r"^(in)([a-z][a-z\-']+)$", token.lower())
    if m:
        return m.group(1), m.group(2)
    return None, None

def _basic_time_from_text(text: str):
    t = text.lower()
    if "tomorrow" in t: return "tomorrow"
    if "today" in t: return "today"
    if "weekend" in t: return "weekend"
    for d in _WEEKDAYS:
        if d in t: return d
    return "today"

def _fuzzy_city_guess(raw: str) -> str | None:
    """
    Fuzzy match the last 1-2 tokens to our lexicon (handles 'sydey' -> 'sydney').
    Also tries the full string first (handles multiword: 'new york').
    """
    if not raw: return None
    q = raw.lower().strip()

    # Try exact multiword first
    if q in _CITY_LEXICON:
        return q

    # Try last 2 tokens window (to catch 'north sydney', 'new york')
    toks = [t for t in re.split(r"\s+", q) if t]
    candidates = []
    if len(toks) >= 2:
        candidates.append(" ".join(toks[-2:]))
    candidates.append(toks[-1])

    for cand in candidates:
        match = difflib.get_close_matches(cand, _CITY_LEXICON, n=1, cutoff=0.75)
        if match:
            # rebuild using matched piece (preserve multiword city if matched)
            return match[0]

    # fallback: fuzzy on the whole raw
    match = difflib.get_close_matches(q, _CITY_LEXICON, n=1, cutoff=0.75)
    return match[0] if match else None

def _classify_attribute(text_lemmas: set, text: str):
    precip_keys = {"rain","umbrella","drizzle","shower","snow","hail","wet"}
    temp_keys   = {"temperature","degree","jacket","coat","cold","hot","warm","freezing","chilly","cool","heat","sweater"}
    wind_keys   = {"wind","windy","gust","storm","breezy","gale"}
    hum_keys    = {"humidity","humid","muggy"}
    cond_keys   = {"weather","forecast","sunny","cloudy","clear","overcast","fog","mist","haze","stormy"}
    sunrise_k   = {"sunrise","dawn"}
    sunset_k    = {"sunset","dusk","twilight"}

    intents = []
    if precip_keys & text_lemmas: intents.append("precipitation")
    if temp_keys   & text_lemmas: intents.append("temperature")
    if wind_keys   & text_lemmas: intents.append("wind")
    if hum_keys    & text_lemmas: intents.append("humidity")
    if sunrise_k   & text_lemmas: intents.append("sunrise")
    if sunset_k    & text_lemmas: intents.append("sunset")
    if cond_keys   & text_lemmas: intents.append("conditions")

    t = text.lower().strip()
    casual = any(kw in t for kw in ["do i need","should i","do i bring","need a","pls","please","?","what should i wear","wear","sweater","jacket","coat"])
    order = ["precipitation","temperature","humidity","wind","sunrise","sunset","conditions"]
    primary = next((i for i in order if i in intents), "temperature")

    wants_umbrella = ("umbrella" in t) or ("rain" in t) or ("wet" in t)
    wants_jacket   = ("jacket" in t) or ("coat" in t) or ("cold" in t) or ("chilly" in t) or ("sweater" in t)

    return primary, intents, {"casual": casual, "wants_umbrella": wants_umbrella, "wants_jacket": wants_jacket}

def smart_parse_weather_question(question: str):
    """
    Returns: {"location","attribute","time_period","style"}
    Robust to 'inkathmandu', 'rainin sydney', misspellings like 'sydey'.
    """
    q = (question or "").strip()
    if not q:
        return {"location": "Perth", "attribute": "temperature", "time_period": "today", "style": {"casual": False}}

    # ---------- Try spaCy if available ----------
    nlp = _get_nlp()
    if nlp:
        doc = nlp(q)
        # 1) Named entities for place
        raw_loc = None
        for ent in doc.ents:
            if ent.label_ in ("GPE","LOC","FAC"):
                raw_loc = ent.text
                break

        # 2) Recover glued 'inCity' pieces
        if not raw_loc:
            for tok in q.split():
                _, maybe_city = _maybe_split_in_stuck(tok)
                if maybe_city:
                    raw_loc = maybe_city
                    break

        # 3) Fallback: regex 'in <city-ish>'
        if not raw_loc:
            m = re.search(r"\bin\s*([a-z][a-z\-\.' ]+)\b", q.lower())
            if m:
                raw_loc = m.group(1)

        # 4) Clean location & fuzzy-correct
        cleaned = _clean_loc_piece(raw_loc or "")
        loc = _fuzzy_city_guess(cleaned) or (cleaned if cleaned else None)

        # 5) Time
        date_text = None
        for ent in doc.ents:
            if ent.label_ == "DATE":
                date_text = ent.text
                break
        time_period = None
        if date_text and dateparse:
            dt = dateparse(date_text)
            if dt:
                today = datetime.now().date()
                d = dt.date()
                if d == today: time_period = "today"
                elif d == today + timedelta(days=1): time_period = "tomorrow"
                elif d.weekday() in (5,6): time_period = "weekend"
                elif 0 <= (d - today).days <= 6: time_period = _WEEKDAYS[d.weekday()]
        if not time_period:
            time_period = _basic_time_from_text(q)

        # 6) Attribute(s)
        lemmas = {t.lemma_.lower() for t in doc if t.is_alpha}
        primary_attr, _all, style = _classify_attribute(lemmas, q)

        return {
            "location": (loc or "Perth").title(),
            "attribute": primary_attr,
            "time_period": time_period,
            "style": style
        }

    # ---------- Fallback: no spaCy ----------
    raw_loc = None
    m = re.search(r"\bin\s*([a-z][a-z\-\.' ]+)\b", q.lower())
    if m:
        raw_loc = m.group(1)
    if not raw_loc:
        for tok in q.split():
            _, maybe_city = _maybe_split_in_stuck(tok)
            if maybe_city:
                raw_loc = maybe_city
                break

    cleaned = _clean_loc_piece(raw_loc or "")
    loc = _fuzzy_city_guess(cleaned) or (cleaned if cleaned else None) or "Perth"
    time_period = _basic_time_from_text(q)

    t = q.lower()
    if any(w in t for w in ["rain","umbrella","drizzle","shower","snow","wet"]):
        attr = "precipitation"
    elif any(w in t for w in ["humidity","humid","muggy"]):
        attr = "humidity"
    elif any(w in t for w in ["wind","windy","gust","storm","breezy","gale"]):
        attr = "wind"
    elif any(w in t for w in ["sunrise","dawn"]):
        attr = "sunrise"
    elif any(w in t for w in ["sunset","dusk","twilight"]):
        attr = "sunset"
    elif any(w in t for w in ["weather","forecast","sunny","cloudy","clear","overcast","fog","mist","haze","stormy"]):
        attr = "conditions"
    else:
        attr = "temperature"

    style = {
        "casual": any(kw in t for kw in ["do i need","should i","do i bring","need a","pls","please","?","what should i wear","wear","sweater","jacket","coat"]),
        "wants_umbrella": ("umbrella" in t) or ("rain" in t) or ("wet" in t),
        "wants_jacket":  ("jacket" in t) or ("coat" in t) or ("cold" in t) or ("chilly" in t) or ("sweater" in t),
    }

    return {
        "location": loc.title(),
        "attribute": attr,
        "time_period": time_period,
        "style": style
    }
# ===================== END SMART NLP PARSER =====================


## 🧭 User Interface

In [12]:
import pyinputplus as pyip
import math
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import requests
from urllib.parse import quote

def main_menu():
    example_keywords = ['example questions', 'example', 'examples', 'show examples', 'sample', 'test', 'q', '3']
    print("="*40)
    print("🌤️ Welcome to WeatherWise!")
    print("Type your question, pick a menu number, or type 'example' anytime to see sample questions.")
    print("="*40)

    while True:
        try:
            raw_choice = pyip.inputStr(
                "\nMain Menu:\n"
                "- [1] Weather Forecast\n"
                "- [2] Ask Question\n"
                "- [3] Example Questions (type 'example')\n"
                "- [4] Exit\n"
                "Your choice: "
            ).strip().lower()

            # ----- Option 1: Direct Forecast -----
            if raw_choice in ('1', 'weather forecast', 'forecast'):
                location = pyip.inputStr("Enter city or location: ")
                if not location.strip():
                    print("Please enter a city name.")
                    continue
                data = get_weather_data(location)
                if isinstance(data, dict) and "error" in data:
                    print(f"[Error] {data['error']}")
                    continue

                header_loc = data.get('display_location') or data.get('requested_location') or location
                print(f"\nForecast for {header_loc}:")
                create_temperature_visualisation(data)
                create_precipitation_visualisation(data)
                create_wind_visualisation(data)
                create_humidity_visualisation(data)

            # ----- Option 2: Ask Question (NLP) -----
            elif raw_choice in ('2', 'ask question', 'ask', 'question'):
                q = pyip.inputStr("Type your weather question: ")
                if not q.strip():
                    print("Please ask a question or choose an option.")
                    continue

                # Parse user's natural question
                pq = smart_parse_weather_question(q)

                print("\n🧠 Parsed intent:")
                print(f"  • Location   : {pq.get('location')}")
                print(f"  • Attribute  : {pq.get('attribute')}")
                print(f"  • Time period: {pq.get('time_period')}")

                ask_loc = pq.get("location")
                data = get_weather_data(ask_loc)

                # Improved error message (user-friendly)
                if isinstance(data, dict) and "error" in data:
                    print("\nSorry — I couldn't get that. Here's what you asked:")
                    print(f'  "{q.strip()}"')
                    print(f"Reason: {data['error']}")
                    print("Try rephrasing, e.g., 'Will it rain in Sydney tomorrow?' or 'What's the temperature in Perth today?'\n")
                    continue

                print("\nHere's what I found:")
                print(generate_weather_response(pq, data))

            # ----- Option 3: Example questions -----
            elif raw_choice in example_keywords:
                print("\nSome things you can ask:")
                print("- Will it rain in Darwin this weekend?")
                print("- Show me the temperature for Perth tomorrow.")
                print("- What's the forecast in Brisbane today?")
                print("- Is it windy in Melbourne right now?")
                print("- Will I need an umbrella in Hobart on Friday?")

            # ----- Option 4: Exit -----
            elif raw_choice in ('4', 'exit', 'quit', 'bye'):
                print("\nThank you for using WeatherWise. Stay safe!")
                break

            # ----- Invalid input -----
            else:
                print("Not a valid option. Type 'example' to see what you can ask, or choose 1, 2, 3, or 4.")

        except KeyboardInterrupt:
            print("\nExiting — stay safe!")
            break
        except Exception as e:
            print(f"[Unexpected error] {e}")


## 🧩 Main Application Logic

In [ ]:
# Tie everything together here


if __name__ == "__main__":
    main_menu()

🌤️ Welcome to WeatherWise!
Type your question, pick a menu number, or type 'example' anytime to see sample questions.

Main Menu:
- [1] Weather Forecast
- [2] Ask Question
- [3] Example Questions (type 'example')
- [4] Exit
Your choice: 2
Type your weather question: will it rain tomorrow in kathmandu

🧠 Parsed intent:
  • Location   : Kathmandu
  • Attribute  : precipitation
  • Time period: tomorrow

Here's what I found:

Main Menu:
- [1] Weather Forecast
- [2] Ask Question
- [3] Example Questions (type 'example')
- [4] Exit
Your choice: 

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

# Sample test
# Quick smoke test
sample = get_weather_data('Melbourne')
if isinstance(sample, dict) and "error" not in sample:
    create_temperature_visualisation(sample)
    create_precipitation_visualisation(sample)
    pq = parse_weather_question("Will it rain tomorrow in Paris?")
    pdata = get_weather_data(pq['location'])
    print(generate_weather_response(pq, pdata))
else:
    print("Fetch error:", sample)



## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.